# Anchor-based Sparse Subspace Incomplete Multi-view Clustering


![image.png](image.png)


### Symbol description

- $A^{(v)} \in R^{n_a \times d^{(v)}}$ is the view-complete subset for each view. Each sample  has all the views. Its size is $n_a$.
- $U^{(v)} \in R^{n^{(v)}_u \times d^{(v)}}$ is the incomplete subset for each view. Each sample only has parts of the views. Its size is $n^{(v)}_u, v = 1,2,\cdots,V$.
- $n^{(v)}$ is the number of existent samples of each view.
- $n_a$ is the number of samples of the complete subset.
- $n^{(v)}_u$ is the number of samples of the incomplete subset of each view.
- $d^{(v)}$ is the original feature dimension of each view.
- $n$ is the total number of samples.
- $V$ is the number of views.
- $c$ is the number of clusters, which is given by the user.

### Inputs

- $ {X}^{(v)} \in R^{ n \times d^{(v)} } $ are all the samples of each view, where inexistent samples are marked with NaN.
- $\bar{X}^{(v)} = [A^{(v)} ; U^{(v)}] \in R^{ n^{(v)} \times d^{(v)} }$ are the reorganized samples of each view, where inexistent samples are dropped and the complete subset is arranged before the incomplete subset.
- $W^{(v)} \in \{0,1\}^{n \times n^{(v)}} $ is the missing indicator matrix.
- $\lambda$ is the balancing parameter in the loss function.

$$
W^{(v)}_{i,j} = \begin{cases}
1\text{, if the $i$-th sample is the $j$-th sample presented in $v$-th view}; \\
0\text{, otherwise}.
\end{cases}
$$

$$
{X}^{(v)} = W^{(v)} \bar{X}^{(v)}
$$

$$
n^{(v)} = \left( n_a + n^{(v)}_u \right) < n\\
v = 1,2,\cdots,V
$$

### Outputs

- $ Z \in R^{ n \times d^{(v)} } $ is the complete consensus anchor graph learned by our method.
- $ C \in \{0,1\}^{n \times c} $ are the clustering results；

## Learning an inter-view consensus sparse subspace matrix for the complete subset

Since each sample of the complete subset possesses all the view, the learned $Z_a$ is the consensus subspace cross the views, which fuses information from different views.

$$
\min \sum_{v=1}^V \left\Vert Z_a^{(v)} A^{(v)} - A^{(v)} \right\Vert _F^2 + \lambda \left\Vert Z_a^{(v)} - Z_a \right\Vert _F^2 \\
\text{s.t. } Z_a^{(v)} \geq 0, Z_a^{(v)} 1 = 1, diag(Z_a^{(v)}) = 0 \\
             Z_a \geq 0, Z_a 1 = 1, diag(Z_a) = 0 \\
$$

- $A^{(v)}$ is the complete subset of view $v$.
- $Z_a^{(v)} \in R^{n_a \times n_a} $ is the subspace matrix of the complete subset of view $v$ constructed by self-representative property.
- $Z_a \in R^{n_a \times n_a}$ is the consensus subspace matrix of the complete subset, that is, the centroid of $\{Z_a^{(v)}\}_{v=1}^V$ .

## Learning the intra-view anchor-based subspace matrices for the incomplete subset

- Since the samples of the incomplete subset are not perfectly aligned, no inter-view consensus subspace can be learned.
- Yet within each view, we can linearly combine samples of $A^{(v)}$ to approximate $U^{(v)}$ so as to learn their relationships as $Z_u^{(v)} \in R^{{n_u}^{(v)} \times n_a} $.
- Then, concatenate two subspace matrices $Z_a, Z_u^{(v)}$ within each view to obtain the anchor graph (bipartile graph) of that view $Z^{(v)} \in R^{{n}^{(v)} \times n_a} $.

$$
\min \sum_{v=1}^V \left\Vert Z_u^{(v)} A^{(v)} - U^{(v)} \right\Vert _F^2 \\
\text{s.t. } Z_u^{(v)} \geq 0, Z_u^{(v)} 1 = 1 \\
$$

## Leanring a anchor graph for view existent samples and a complete consensus anchor graph

Putting the above 1. inter-view consensus sparse subspace learning and 2. intra-view anchor-based sparse subspace learning into a unified optimization problem.

$$
\min \sum_{v=1}^V \left\{ \left\Vert Z_a^{(v)} A^{(v)} - A^{(v)} \right\Vert _F^2 +
\lambda \left\Vert Z_a^{(v)} - Z_a \right\Vert _F^2 +
\left\Vert Z_u^{(v)} A^{(v)} - U^{(v)} \right\Vert _F^2 \right\} \\
\text{s.t. } Z_a^{(v)} \geq 0, Z_a^{(v)} 1 = 1, diag(Z_a^{(v)}) = 0 \\
             Z_a \geq 0, Z_a 1 = 1, diag(Z_a) = 0 \\
             Z_u^{(v)} \geq 0, Z_u^{(v)} 1 = 1
$$

- Compute the anchor graph of the existent samples of each view as: $Z^{(v)} = [ Z_a ; Z_u^{(v)} ] \in R^{n^{(v)} \times n_a} $.
- Compute the complete consensus anchor graph as: $Z = \frac{1}{V} \sum_{v=1}^V W^{(v)} Z^{(v)}$ .
- Perform Fast Spectral Clustering on $Z$ and obtain the final clustering results $C$ .

## Equivalent unconstrained loss and its optimization

We transform the above objective function into an equivalent unconstrained form to take advantage of autograd and hardware acceleration of PyTorch. 

- 用掩码softmax激活函数$\sigma_m(X, M)$实现约束条件 $Z \geq 0, Z 1 = 1, diag(Z)=0$，即：
  - $Z$的元素大于等于0；
  - $Z$的每行元素之和都为1；
  - $Z$的对角线元素为0；
- 使用均方误差损失 $\mathcal{L}_{MSE}(X, Y)$ 来实现F范数误差，即$\left\Vert X - Y \right\Vert_F^2$；
- 待求的子空间矩阵 $\{Z_a,Z_a^{(v)},Z_u^{(v)}\}_{v=1}^V$ 分别用可学习参数 $\{\Theta_a,\Theta_a^{(v)},\Theta_u^{(v)}\}_{v=1}^V$ 表示；
- 使用Adam优化算法最小化损失函数 $\mathcal{L}_{\text{ABSS-IMC}}$ 即可；

$$
\mathcal{L}_{\text{ABSS-IMC}} = 
\sum_{v=1}^V \left\{ \mathcal{L}_{MSE} \left( \sigma_m(\Theta_a^{(v)}, M_{n_a}) A^{(v)}, A^{(v)} \right) +
\lambda \mathcal{L}_{MSE} \left( \Theta_a^{(v)} , \Theta_a \right) +
\mathcal{L}_{MSE} \left( \sigma(\Theta_u^{(v)}) , U^{(v)} \right)
\right\}
$$

$$
[\sigma_m(X, M)]_{i,j} = \frac{\exp(X_{i,j}) \cdot M_{i,j}}{\sum_{j=1}^n \exp(X_{i,j}) \cdot M_{i,j}}
$$

## 前后处理

- 前处理：在优化损失函数之前，对输入进行前处理；
- 后处理：在优化损失函数之后，学到了参数$\{\Theta_a,\Theta_a^{(v)},\Theta_u^{(v)}\}_{v=1}^V$，再进行后处理以获得聚类结果$C$和完整共识锚点图$Z$；

### Preprocessing steps

- Normalize each feature to a real number in [0, 1];
- Reorganize the normalized features into the form of $\bar{X}^{(v)} = [A^{(v)} ; U^{(v)}]$, where:
  - $A^{(v)} \in R^{n_a \times d^{(v)}}$ is the complete subset for each view.
  - $U^{(v)} \in R^{n^{(v)}_u \times d^{(v)}}$ is the incomplete subset for each view.
- The relationship between $\bar{X}^{(v)}$ and ${X}^{(v)}$ is: ${X}^{(v)} = W^{(v)} \bar{X}^{(v)}$；

$$
\tilde{X} = \frac{X - \min_i \{X_i\}}{\max_i \{X_i\}}
$$

### Postprocessing steps

- 从可学习参数 $\{\Theta_a,\Theta_u^{(v)}\}_{v=1}^V$ 得到 $\{Z_a,Z_u^{(v)}\}_{v=1}^V$ 如下：
  - $Z_a = \sigma(M \Theta_a)$；
  - $Z_u^{(v)} = \sigma(\Theta_u^{(v)})$；
- 从$Z_a,\{Z_u^{(v)}\}_{v=1}^V$ 得到完整共识锚点图$Z$ 如下：
  - $Z^{(v)} = [ Z_a ; Z_u^{(v)} ] \in R^{n^{(v)} \times n_a} $ ；
  - $Z = \frac{1}{V} \sum_{v=1}^V W^{(v)} Z^{(v)}$ ；
- 对完整共识锚点图$Z$进行快速谱聚类如下：
  - 对$Z$进行奇异值分解，即 $Z=P\Sigma Q^T$；
  - $Z$ 的左奇异向量$P \in R^{n \times c}$即为$Z$的谱嵌入，取原始的左奇异向量$P$的前$c$个特征；
  - 对$P$的模长进行归一化，即$\tilde{P}_i = \frac{P_i}{||P_i||_F}$；
  - 对归一化后的$\tilde{P}_i$进行K-means聚类，得到最终聚类结果$C \in \{0, 1\}^{n \times c}$；

## 实现细节

- Use Adam optimizer with learning rate set to 0.1.
- Hyper-parameter $\lambda$ is set to 0.1.
- Take the first $c$ features of the left singular matrix $P$ and perform $L_2$ normalization on it when doing spectral clustering.